# Data Cleaning
Here, we convert types and units for a more usable data frame. First, import packages and csv.

In [1]:
import pandas as pd
import numpy as np

In [2]:
#load csv file
food_df=pd.read_csv('data/recipes.csv',encoding='utf-8')

## Convert prep time, cook time, and total time to integer (minutes)
It will be more useful to have all of our times be given in minutes, rather than a mix of minutes and hours.

In [3]:
#Convert to minutes
#get hours
hours_series=food_df['CookTime'].replace(to_replace='PT(?:(\d+)H)?(?:(\d+)M)?',value=r"\1", regex=True)
#replace empty string with 0
hours_series.replace(r'^\s*$', 0, regex=True,inplace=True)

#get minutes
minutes_series=food_df['CookTime'].replace(to_replace='PT(?:(\d+)H)?(?:(\d+)M)?',value=r"\2", regex=True)
#replace empty string with 0
minutes_series.replace(r'^\s*$', 0, regex=True,inplace=True)

#replace NaNs (no cooktime)
hours_series[hours_series.isnull()]=0
minutes_series[minutes_series.isnull()]=0

#convert to minutes
food_df['CookTime']=minutes_series.astype(int)+60*hours_series.astype(int)

In [4]:
#Do the same with PrepTime
#two recipes say negative prep time, should be positive
#print(food_df[food_df['PrepTime'].str.contains("-")])
food_df['PrepTime']=food_df['PrepTime'].str.replace('-','')

#Convert to minutes
#get hours
hours_series=food_df['PrepTime'].replace(to_replace='PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?',value=r"\1", regex=True)
#replace empty string with 0
hours_series.replace(r'^\s*$', 0, regex=True,inplace=True)

#get minutes, remove negative entries
minutes_series=food_df['PrepTime'].replace(to_replace='PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?',value=r"\2", regex=True)
#replace empty string with 0
minutes_series.replace(r'^\s*$', 0, regex=True,inplace=True)

#get seconds - Did not end up needing, all were 0.
#seconds_series=food_df['PrepTime'].replace(to_replace='PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?',value=r"\3", regex=True)
#replace empty string with 0
#seconds_series.replace(r'^\s*$', 0, regex=True,inplace=True)


#replace NaNs (no cooktime)
hours_series[hours_series.isnull()]=0
minutes_series[minutes_series.isnull()]=0
#seconds_series[seconds_series.isnull()]=0

#convert to minutes
food_df['PrepTime']=minutes_series.astype(int)+60*hours_series.astype(int)

In [5]:
#Finally, do the same with total time.
#remove negative preptime
#print(food_df[food_df['TotalTime'].str.contains("-")])
food_df['TotalTime']=food_df['TotalTime'].str.replace('-','')

#Convert to minutes
#get hours
hours_series=food_df['TotalTime'].replace(to_replace='PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?',value=r"\1", regex=True)
#replace empty string with 0
hours_series.replace(r'^\s*$', 0, regex=True,inplace=True)

#get minutes
minutes_series=food_df['TotalTime'].replace(to_replace='PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?',value=r"\2", regex=True)
#replace empty string with 0
minutes_series.replace(r'^\s*$', 0, regex=True,inplace=True)

#replace NaNs (no cooktime)
hours_series[hours_series.isnull()]=0
minutes_series[minutes_series.isnull()]=0

#convert to minutes
food_df['TotalTime']=minutes_series.astype(int)+60*hours_series.astype(int)

## Add columns 
We will add columns for number of images, number of ingredients, number of steps, and will clean the keywords column.

### Number of Images.
Index 264469 has "NaN" in images column instead of character(0), and needs to be treated sepatately.

In [6]:
#If the image entry is character(0), no images, returns 0. Else, returns number of images (separated by ", ")
food_df['NumImages']=(food_df['Images']!='character(0)')*(1+food_df['Images'].str.count('", "'))
food_df.loc[264469,'NumImages']=0
food_df['NumImages']=food_df['NumImages'].astype(int)

### Number of Ingredients.

In [7]:
food_df['RecipeIngredientParts']=food_df['RecipeIngredientParts'].str.strip('c').str.strip('(').str.strip(')')
food_df['NumIngredients']=(food_df['RecipeIngredientParts']!='character(0)')*(1+food_df['RecipeIngredientParts'].str.count('", "'))

### Number of Steps.

In [8]:
food_df['RecipeInstructions']=food_df['RecipeInstructions'].str.strip('c').str.strip('(').str.strip(')')
food_df['NumSteps']=(food_df['RecipeInstructions']!='character(0)')*(1+food_df['RecipeInstructions'].str.count('", "'))

### Clean Keywords and Quantities
We get rid of the c() around the keywords and ingredient quantities.

In [9]:
food_df['Keywords']=food_df['Keywords'].str.strip('c').str.strip('(').str.strip(')')
food_df['RecipeIngredientQuantities']=food_df['RecipeIngredientQuantities'].str.strip('c').str.strip('(').str.strip(')')

## DatePublished to DateTime

In [10]:
food_df['DatePublished']=pd.to_datetime(food_df['DatePublished'],format='%Y-%m-%dT%H:%M:%SZ')

In [11]:
pd.set_option("display.max_columns", 100)
food_df.head(20)

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,NumImages,NumIngredients,NumSteps
0,12021,Swirled Cranberry Cheesecake,14386,Divinemom5,120,15,135,2001-09-23 15:35:00,Make and share this Swirled Cranberry Cheeseca...,"""https://img.sndimg.com/food/image/upload/w_55...",Cheesecake,"""Dessert"", ""Cheese"", ""Fruit"", ""Winter"", ""Chris...","""2 1/4"", ""1/4"", ""1"", ""2"", ""1/4"", ""3"", ""1"", ""1""...","""butter"", ""whole berry cranberry sauce"", ""cinn...",5.0,9.0,4575.7,304.1,184.9,1716.5,2756.6,404.4,7.3,376.7,78.2,NaN,1 9-inch cheescake,"""Mix cookie crumbs and butter."", ""Press onto b...",1,9,20
1,348900,Chicken Basquaise,287420,English_Rose,40,20,60,2009-01-11 13:12:00,At the very time you are serving stir in some ...,character(0),Chicken Breast,"""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"", ""...","""1"", ""1"", ""1"", ""1"", ""3"", ""2/3"", ""1 1/4"", ""10"",...","""chicken thighs"", ""chicken breasts"", ""olive oi...",NaN,NaN,1073.2,64.7,18.8,305.3,834.3,36.0,2.7,4.6,69.1,4.0,NaN,"""Put a tablespoon of oil in a medium flameproo...",0,12,7
2,102996,Brandy Alexander Pie,98467,Scout33,120,30,150,2004-10-30 20:00:00,A congealed version of the Frozen Brandy Alexa...,character(0),Gelatin,"""Dessert"", ""< 4 Hours""","""1"", ""1"", ""1/4"", ""1 1/2"", ""1/4"", ""2"", ""2""","""9-inch graham cracker crusts"", ""unflavored ge...",5.0,1.0,352.3,24.0,11.8,61.1,189.8,28.4,0.5,19.1,2.9,NaN,NaN,"""Sprinkle gelatin over cold water in small sau...",0,6,7
3,373435,Creole Sauteed Cabbage,96177,BakinBaby,20,3,23,2009-05-20 20:13:00,Andouille or smoke sausage sauteed with cabbag...,"c(""https://img.sndimg.com/food/image/upload/w_...",Vegetable,"""Creole"", ""< 30 Mins""","""1"", ""1/2"", ""1"", ""1"", ""1"", ""1"", ""1""","""cabbage"", ""andouille sausage"", ""onion"", ""baco...",5.0,5.0,102.4,4.0,2.2,9.0,83.6,15.9,6.0,9.1,3.4,4.0,NaN,"""Fry bacon and sausage in a large skillet abou...",3,6,4
4,299106,Hyderabad Chicken Curry,409106,Brian Holley,70,10,80,2008-04-17 02:13:00,A slightly sweeter chicken curry due to the us...,character(0),Curries,"""Chicken"", ""Poultry"", ""Meat"", ""Asian"", ""Indian...","""1"", ""2"", ""2"", ""2"", ""2"", ""2"", ""2"", ""1"", ""3"", ""...","""onion"", ""ghee"", ""butter"", ""garlic cloves"", ""g...",5.0,1.0,723.2,26.6,12.3,326.0,270.1,10.1,2.6,3.0,107.0,4.0,NaN,"""Skin and joint the chicken, set aside."", ""Hea...",0,14,5
5,15410,Learn Your Abc's,10649,Ben Ross,0,5,5,2001-12-07 09:56:00,This is a very old cocktail. This is one of my...,character(0),Beverages,"""< 15 Mins"", ""No Cook"", ""Easy""","""15"", ""15"", ""15""","""Amaretto"", ""Baileys Irish Cream"", ""cognac""",NaN,NaN,51.6,2.5,1.5,9.2,14.5,3.3,0.0,3.1,0.4,1.0,NaN,"""Layer your A, B and C in a shot glass."", ""Tak...",0,3,3
6,181352,Budbud Kabog Recipe,261327,gailinsik,60,120,180,2006-08-10 14:29:00,Make and share this Budbud Kabog Recipe recipe...,"c(""https://img.sndimg.com/food/image/upload/w_...",Breakfast,"""Grains"", ""Filipino"", ""Asian"", ""For Large Grou...","""3"", NA, NA, ""2"", NA, ""3/4"", ""2""","""fresh coconut"", ""warm water"", ""water"", ""sugar...",4.0,3.0,63.1,4.2,3.6,0.0,49.1,6.2,1.4,2.2,0.8,100.0,NaN,"""Preparation stage:"", ""Grate two of the mature...",6,5,8
7,94316,Turkey Salad Polynesian,129958,Bobtail,0,30,30,2004-06-25 20:00:00,This is another delicious warm weather salad f...,"c(""https://img.sndimg.com/food/image/upload/w_...",Low Cholesterol,"""Healthy"", ""Thanksgiving"", ""< 30 Mins"", ""Begin...","""8"", ""2"", ""3/4"", ""3/4"", ""1/4"", ""1"", ""1"", ""1/2""...","""mostaccioli pasta"", ""celery"", ""walnuts"", ""gre...",5.0,3.0,353.6,16.4,3.3,34.6,290.2,36.6,3.0,9.2,16.7,8.0

## Remove html artifacts.

We have:
+ \&quot; instead of ",
+ \&eacute; instead of &eacute;,
+ \&ntilde; instead of &ntilde;,
+ and \&amp; instead of &,
+ etc.

There is a package that will fix most of this for us. We are left with some artifacts - we deal with some of these remaining artifacts, looking for instances where we have both ";" and "&" leaves us with over 200 entries.

In [12]:
#There is a package that will do this for us:
from bs4 import BeautifulSoup
food_df['Name']=food_df['Name'].map(lambda text: BeautifulSoup(text, 'html.parser').get_text())

#food_df[food_df['Name'].str.contains(";")]

In [13]:
# fixes some html that wasn't replaced
food_df['Name']=food_df['Name'].str.replace('&amp;amp;','&')
food_df['Name']=food_df['Name'].str.replace('&amp;','&')
food_df['Name']=food_df['Name'].str.replace('&quot;','"')
food_df['Name']=food_df['Name'].str.replace('&eacute;','é')
food_df['Name']=food_df['Name'].str.replace('&amp;','&')
food_df['Name']=food_df['Name'].str.replace('&ntilde;','ñ')
food_df['Name']=food_df['Name'].str.replace('&oslash;','ø')
food_df['Name']=food_df['Name'].str.replace('&ccedil;','ç')
food_df['Name']=food_df['Name'].str.replace('&aacute;','á')
food_df['Name']=food_df['Name'].str.replace('&uacute;','ú')
food_df['Name']=food_df['Name'].str.replace('&uacute;','ú')
food_df['Name']=food_df['Name'].str.replace('&ndash;','–')
food_df['Name']=food_df['Name'].str.replace('&agrave;','à')

#fix apostrophe's
food_df['Name']=food_df['Name'].str.replace('&#039;','\'')

In [14]:
food_df.loc[51607,'Name']


"Spinach Pasta Salad Recipe | Kathy's Vegan Kitchen"

In [15]:
print(food_df[(food_df['Name'].str.contains(";"))&(food_df['Name'].str.contains("&"))].size)
food_df[(food_df['Name'].str.contains(";"))&(food_df['Name'].str.contains("&"))]

248


,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,NumImages,NumIngredients,NumSteps
12909,421951,Yummy Korean Seafood Pancakes Haemul Pajeon (&...,1604498,donoteatme,10,10,20,2010-04-27 10:38:00,These seafood pancakes have been my favorite e...,character(0),Onions,"""Peppers"", ""Vegetable"", ""Korean"", ""Asian"", ""We...","""1"", ""1/4"", ""1"", ""2"", ""6"", ""5 -6"", ""5"", ""1"", ""...","""all-purpose flour"", ""rice powder"", ""cold wate...",NaN,NaN,632.9,11.5,3.4,423.0,149.6,103.3,5.8,6.1,26.7,1.0,1 pancake,"""In a large bowl mix flour, sweet rice powder,...",0,11,3
31962,254886,Pumpkin Soup With Red Cabbage and Cumin Seeds ...,594771,blinkinglemon,30,20,50,2007-09-24 19:26:00,Make and share this Pumpkin Soup With Red Cabb...,character(0),Vegetable,"""Turkish"", ""Southwest Asia (middle East)"", ""As...","""3"", ""1"", ""3"", ""3"", ""1/2"", ""1"", ""3"", ""1/2"", ""1...","""pumpkin"", ""onion"", ""olive oil"", ""garlic clove...",NaN,NaN,164.6,11.9,2.2,4.0,22.2,13.9,2.5,4.7,3.3,4.0,NaN,"""We cut the pumpkin you see above into two, to...",0,14,3
126226,157753,P&#260;czki (Polish Doughnuts),200356,Anka4505,120,20,140,2006-02-27 18:34:00,Make and share this P&amp;#260;czki (Polish Do...,character(0),Breads,"""European"", ""Hanukkah"", ""Brunch"", ""For Large G...","""500"", ""5"", ""3"", ""1"", ""1"", ""5"", ""1"", ""50"", ""1""...","""all-purpose flour"", ""dry yeast"", ""egg"", ""nonf...",NaN,NaN,159.7,9.4,2.0,29.7,17.6,15.6,0.6,2.6,2.7,30.0,NaN,"""Combine yeast with half a cup of warm milk, a...",0,8,13
152162,189503,Sparkling Wine Cocktail; Raspberries & Chambord,232669,Manami,120,15,135,2006-10-08 20:22:00,Make and share this Sparkling Wine Cocktail; ...,character(0),Beverages,"""Raspberries"", ""Berries"", ""Fruit"", ""Brunch"", ""...","""1/2"", ""3 -4"", ""2"", ""12 2/3""","""fresh raspberry"", ""Chambord raspberry liquor""...",NaN,NaN,187.2,0.2,0.0,0.0,9.6,12.7,2.0,7.3,0.5,2.0,NaN,"""Gently stir together raspberries, Chambord, a...",0,3,4
177548,179331,Better Than &#8219;beef’ Stroganoff,290499,Vegan Freak,20,15,35,2006-07-26 11:44:00,Short on time but looking for a hearty meal? T...,character(0),Vegetable,"""Lactose Free"", ""Vegan"", ""Egg Free"", ""Free Of....","""1"", ""1/2"", ""1"", ""1"", ""1"", ""2"", ""1/8"", ""1/4"", ...","""onion"", ""garlic powder"", ""pepper"", ""salt"", ""v...",NaN,NaN,292.6,12.4,1.6,0.0,1114.9,23.4,6.5,3.6,23.4,4.0,NaN,"""Cook the pasta in boiling water until the des...",0,5,3
369360,465368,Sausage & Sage Pierogies; the Heaping Bowl,772128,madriley61,10,5,15,2011-10-02 14:54:00,This recipe is from a restaurant in East Atlan...,character(0),One Dish Meal,"""Pork"", ""Potato"", ""Vegetable"", ""Meat"", ""Polish...","""8"", ""2"", ""2"", ""8"", ""16"", ""4"", ""4"", ""1"", ""1"", ...","""mild Italian sausage"", ""granny smith apples"",...",NaN,NaN,181.2,9.6,5.8,28.6,55.5,17.4,2.5,11.0,3.4,4.0,NaN,"""Heat 1 tablespoon vegetable oil or butter in ...",0,8,4
394399,390718,Korean Seasoned Potatoes (&#44048;&#51088; &#5...,93698,Andy Wold,0,20,20,2009-09-17 14:44:00,Make and share this Korean Seasoned Potatoes (...,"""https://img.sndimg.com/food/image/upload/w_55...",Potato,"""Vegetable"", ""Korean"", ""Asian"", ""Low Protein"",...","""2"", ""2"", ""2"", ""1"", ""2"", ""1 -2"", ""4""","""potatoes"", ""soy sauce"", ""sugar"", ""garlic"", ""s...",4.5,2.0,117.0,2.0,0.3,0.0,510.4,22.2,2.8,3.2,3.5,4.0,NaN,"""Peel potatoes and dice, placing in cold water...",1,6,5
413344,399933,"Squash, Chickpea & Red Lentil Stew (Healthy; S...",890605,GoldsmithLissa,600,20,620,2009-11-17 20:28:00,"Make and share this Squash, Chickpea &amp; Red...",character(0),One Dish Meal,"""Vegetable""","""

In [16]:
#Turn Keywords into a list
food_df['KeywordsTemp']=food_df['Keywords'].str.replace('", ',',')
food_df['KeywordsTemp']=food_df['Keywords'].str.replace('"','')

food_df['KeywordsTemp']=food_df['KeywordsTemp'].astype(str).str.split(", ")

food_df['Keywords']=food_df['KeywordsTemp']
food_df=food_df.drop(columns=['KeywordsTemp'])

In [18]:
#Turn Ingredients into a list
food_df['RecipeIngredientPartsTemp']=food_df['RecipeIngredientParts'].str.replace('", ',',')
food_df['RecipeIngredientPartsTemp']=food_df['RecipeIngredientParts'].str.replace('"','')

food_df['RecipeIngredientPartsTemp']=food_df['RecipeIngredientPartsTemp'].astype(str).str.split(", ")

#food_df['RecipeIngredientParts']=food_df['RecipeIngredientPartsTemp']
#food_df=food_df.drop(columns=['RecipeIngredientPartsTemp'])